# Stress Detection Notebook

Algorithm based on: <br>
`M. Salai, I. Vassanyi, and I. Kosa, “Stress Detection Using Low Cost Heart Rate Sensors,” Journal of Healthcare Engineering,  2016.`

With incorporating of the physical activity index from: <br>
`J. Bai,et al., “An Activity Index for Raw Accelerometer Data and Its Comparison with Other Activity Metrics,” PLoS ONE, vol. 11, 2016.`

In [121]:
import sys
import getpass
from pathlib import Path
from collections import Counter

import pandas as pd
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt

import pyedflib
from pandas.compat import StringIO

import heartpy as hp

In [2]:
user = getpass.getuser()
data_path = '/Users/{}/GitHub/fydp-data-science/data/ondri_data/'.format(user)

## Setting variables
- variables are based on the values presented in the Salai paper

In [3]:
variables = {
    'sampling_frequency': 1000,
    'window_size': 560,
    'shift': 20,
    'n_intervals': 4,
}

thresholds = {
    'mean_bpm_change': 5,
    'rmssd_change': -9,
    'pnn50_change': -9,
    'activity_intensity': 'low'
}

In [29]:
sampling_rate = 100

## Methods to detect stress

#### Calculating percent difference
- so we can compare how HRV features change between intervals
- inputs are the HRV features from the respective intervals:

In [5]:
def find_percent_difference(first_val, second_val):
   
    diff = ((second_val - first_val) / first_val)*100
    
    return diff

#### Detecting stress
- inputs: variables and threholds from the Salai paper, as well as the processed ECG signal and physical activity intensities
- this returns an array of detected stress events

In [127]:
def detect_stress(data, variables, thresholds):
    
    # process data using heartpy
    ecg_data, ecg_measures = hp.process(data.ECG, sampling_rate)
    
    # prepping some variables
    # find the first peak and final peak of the window
    # start of the window: first peak (RR wave)
    # end of the window: 560th peak (RR wave)
    s = 0
    e = variables['window_size']
    start_index = ecg_data['peaklist'][s]
    end_index = ecg_data['peaklist'][e]
    
    # instantiate stress related variables
    stress_event = False
    stress_results = []
    
    # looping through dataset to roll the detection window
    while end_index <= ecg_data['peaklist'][-1]:
        
        # grab window
        window = ecg_data['hr'][start_index:end_index].values
        
        # split window into n intervals
        intervals = []
        for x in range(0, len(window), int(len(window)/variables['n_intervals'])):
            # we don't want to grab more intervals than expected
            if len(intervals) < variables['n_intervals']:
                intervals.append(window[x : x + int(len(window)/variables['n_intervals'])])
        
        # instantiate variables for HRV measures
        mean_bpm = []
        rmssd = []
        pnn50 = []
        
        # interating through the intervals to calculate the HRV features
        for i in intervals:
            # processing the interval dataset using heartpy to find HRV features
            interval_data, measures = hp.process(i, variables['sampling_frequency'])
            
            # appending HRV features to their respective arrays
            mean_bpm.append(measures['bpm'])
            rmssd.append(measures['rmssd'])
            pnn50.append(measures['pnn50'])
            
        # compare hrv features across intervals. We are comparing the following:
        # mean bpm: change between intervals 1 and 4
        # rmssd: change between intervals 3 and 4
        # pnn50: change between intervals 3 and 4
        mean_bpm_diff = find_percent_difference(mean_bpm[0], mean_bpm[variables['n_intervals']-1])
        rmssd_diff = find_percent_difference(rmssd[variables['n_intervals']-2], rmssd[variables['n_intervals']-1])
        pnn50_diff = find_percent_difference(pnn50[variables['n_intervals']-2], pnn50[variables['n_intervals']-1])
        
        # grabbing the physical activity intensity data
        # looking at what activity intensity the majority of the interval was in
        activity_intensity = max(Counter(data.level[start_index:end_index]))
        
        # detecting stress depending on how features change
        # features change in a manner that depicts stress event
        if ((mean_bpm_diff > threshold['mean_bpm_change']) and (rmssd_diff < threshold['rmssd_change']) 
            and (pnn50_diff < threshold['pnn50_change']) and (activity_intensity == threshold['activity_intensity'])):
            stress_event = True

        # features change in a manner that depicts end of stress event
        elif ((mean_bpm_diff < -threshold['mean_bpm_change']) and (rmssd_diff > -threshold['rmssd_change']) 
              and (pnn50_diff > -threshold['pnn50_change']) and (activity_intensity != threshold['activity_intensity'])):
            stress_event = False
            
        # push stress result to array
        stress_results.append(stress_event)

        # update window indices
        s = s + variables['shift']
        e = e + variables['shift']

        # maybe adding an edge case here would be a good idea in order to detect in the final 560RR of the dataset
        start_index = hr_data['peaklist'][s]
        end_index = hr_data['peaklist'][e]
        
    return stress_results

## Pulling in data

In [21]:
data = pd.read_csv(data_path + 'combined_ecg_activity_data.csv', index_col=0)

# clip the data as needed
data = data[200000:400000].reset_index().drop('index',1)

data.head()

,time,ECG,level
0,400000,117.0,low
1,400001,115.0,low
2,400002,106.0,low
3,400003,101.0,low
4,400004,96.0,low


## Running code

In [120]:
stress_results = detect_stress(data, variables, thresholds)
stress_results

75 33590
[array([978, 978, 977, ..., 414, 411, 405]), array([394, 383, 371, ..., 672, 689, 696]), array([695, 684, 664, ..., 473, 489, 509]), array([535, 566, 603, ..., 589, 628, 661])]
[978 978 977 ... 414 411 405]


BadSignalWarning: 
----------------
Could not determine best fit for given signal. Please check the source signal.
 Probable causes:
- detected heart rate falls outside of bpmmin<->bpmmax constraints
- no detectable heart rate present in signal
- very noisy signal (consider filtering and scaling)
If you're sure the signal contains heartrate data, consider filtering and/or scaling first.
----------------
